## 자연어처리-이병남-1197

## 답안지
[1] 3
[2] 2
[3] 3
[4] 2
[5] 3
[6] 3
[7] 1
[8] 3
[9] 2
[10] 1

### 실습1

In [4]:
# 필수 라이브러리 Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Scikit-Learn 모듈들
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [8]:
# 🚗 MPG 자동차 연비 데이터 로드
print("🔄 MPG 자동차 연비 데이터를 로드하고 있습니다...")

try:
    # MPG CSV 파일 로드
    mpg_data = pd.read_csv('mpg.csv', encoding='utf-8')
    print("✅ MPG 데이터를 성공적으로 로드했습니다!")
    
except FileNotFoundError:
    print("❌ mpg.csv 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    # 대안으로 샘플 데이터 생성
    print("⚠️ 샘플 MPG 데이터를 생성합니다...")
    
    # 샘플 데이터 생성
    sample_data = {
        'manufacturer': ['audi', 'audi', 'audi', 'chevrolet', 'dodge'] * 20,
        'model': ['a4', 'a4', 'a4 quattro', 'malibu', 'caravan 2wd'] * 20,
        'displ': [1.8, 2.0, 2.8, 2.4, 3.3] * 20,
        'year': [1999, 2008, 1999, 2008, 1999] * 20,
        'cyl': [4, 4, 6, 4, 6] * 20,
        'trans': ['auto(l5)', 'manual(m6)', 'auto(l5)', 'auto(l4)', 'auto(l4)'] * 20,
        'drv': ['f', 'f', '4', 'f', 'f'] * 20,
        'cty': [18, 20, 16, 22, 17] * 20,
        'hwy': [29, 31, 26, 30, 24] * 20,
        'fl': ['p', 'p', 'p', 'r', 'r'] * 20,
        'category': ['compact', 'compact', 'compact', 'midsize', 'minivan'] * 20
    }
    mpg_data = pd.DataFrame(sample_data)
    
except Exception as e:
    print(f"❌ 데이터 로드 중 오류 발생: {e}")
    raise

print(f"✅ MPG 자동차 연비 데이터 준비 완료!")
print(f"📏 데이터 크기: {mpg_data.shape}")
print(f"📊 컬럼 목록: {list(mpg_data.columns)}")

# 기본 통계 정보
if 'cty' in mpg_data.columns:
    print(f"🏙️ 시내 연비 범위: {mpg_data['cty'].min():.1f} ~ {mpg_data['cty'].max():.1f} mpg")
if 'hwy' in mpg_data.columns:
    print(f"🛣️ 고속도로 연비 범위: {mpg_data['hwy'].min():.1f} ~ {mpg_data['hwy'].max():.1f} mpg")
if 'year' in mpg_data.columns:
    print(f"📅 연식 범위: {mpg_data['year'].min()} ~ {mpg_data['year'].max()}")

# 데이터 구조 확인
print(f"\n📊 데이터 구조:")
print(mpg_data.info())

print(f"\n📈 이미지와 동일한 형식으로 데이터프레임 표시:")
print("=" * 80)
# 문제와 동일한 형식으로 첫 10개 행 표시
display_df = mpg_data.head(10).copy()
display_df.index = range(len(display_df))  # 인덱스를 0부터 시작하도록 설정
display(display_df)

print(f"\n🔍 제조사별 차량 수:")
manufacturer_counts = mpg_data['manufacturer'].value_counts()
print(manufacturer_counts)

print(f"\n🔍 차량 카테고리별 분포:")
category_counts = mpg_data['category'].value_counts()
print(category_counts)

🔄 MPG 자동차 연비 데이터를 로드하고 있습니다...
❌ mpg.csv 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.
⚠️ 샘플 MPG 데이터를 생성합니다...
✅ MPG 자동차 연비 데이터 준비 완료!
📏 데이터 크기: (100, 11)
📊 컬럼 목록: ['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty', 'hwy', 'fl', 'category']
🏙️ 시내 연비 범위: 16.0 ~ 22.0 mpg
🛣️ 고속도로 연비 범위: 24.0 ~ 31.0 mpg
📅 연식 범위: 1999 ~ 2008

📊 데이터 구조:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  100 non-null    object 
 1   model         100 non-null    object 
 2   displ         100 non-null    float64
 3   year          100 non-null    int64  
 4   cyl           100 non-null    int64  
 5   trans         100 non-null    object 
 6   drv           100 non-null    object 
 7   cty           100 non-null    int64  
 8   hwy           100 non-null    int64  
 9   fl            100 non-null    object 
 10  category      100 non-null    object 

manufacturer        model  displ  year  cyl       trans drv  cty  hwy fl  \
0         audi           a4    1.8  1999    4    auto(l5)   f   18   29  p   
1         audi           a4    2.0  2008    4  manual(m6)   f   20   31  p   
2         audi   a4 quattro    2.8  1999    6    auto(l5)   4   16   26  p   
3    chevrolet       malibu    2.4  2008    4    auto(l4)   f   22   30  r   
4        dodge  caravan 2wd    3.3  1999    6    auto(l4)   f   17   24  r   
5         audi           a4    1.8  1999    4    auto(l5)   f   18   29  p   
6         audi           a4    2.0  2008    4  manual(m6)   f   20   31  p   
7         audi   a4 quattro    2.8  1999    6    auto(l5)   4   16   26  p   
8    chevrolet       malibu    2.4  2008    4    auto(l4)   f   22   30  r   
9        dodge  caravan 2wd    3.3  1999    6    auto(l4)   f   17   24  r   

  category  
0  compact  
1  compact  
2  compact  
3  midsize  
4  minivan  
5  compact  
6  compact  
7  compact  
8  midsize  
9  minivan


🔍 제조사별 차량 수:
manufacturer
audi         60
chevrolet    20
dodge        20
Name: count, dtype: int64

🔍 차량 카테고리별 분포:
category
compact    60
midsize    20
minivan    20
Name: count, dtype: int64


In [ ]:
### 실습 2

In [10]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("mpg.csv")

# Filter rows where manufacturer is 'audi'
audi_df = df[df["manufacturer"] == "audi"]

# Display the result
print(audi_df)


FileNotFoundError: [Errno 2] No such file or directory: 'mpg.csv'

### 실습 3

In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("mpg.csv")

# 각 열을 변수에 저장
category = df["category"]
cty = df["cty"]
hwy = df["hwy"]

# 예시 출력
print(category.head())
print(cty.head())
print(hwy.head())


### 실습 4

In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("mpg.csv")

# category별로 cty, hwy 평균 계산
avg_by_category = df.groupby("category")[["cty", "hwy"]].mean()

# 결과 출력
print(avg_by_category)


### 실습 5

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# CSV 파일 불러오기
df = pd.read_csv("mpg.csv")

# category별로 cty, hwy 평균 계산 후 chart 변수에 저장
chart = df.groupby("category")[["cty", "hwy"]].mean()

# 선 그래프 그리기
plt.figure(figsize=(8, 5))
plt.plot(chart.index, chart["cty"], marker='o', label='도시 연비 (cty)', color='blue')
plt.plot(chart.index, chart["hwy"], marker='s', label='고속도로 연비 (hwy)', color='green')

# 그래프 꾸미기
plt.title("차종별 평균 연비")
plt.xlabel("차종 (category)")
plt.ylabel("연비 (mpg)")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()

# 그래프 출력
plt.show()
